In [11]:
reset -fs

In [12]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point, Polygon
from pyproj import CRS
import folium


## Example polygon

In [13]:
# lat_point_list = [50.854457, 52.518172, 50.072651, 48.853033, 50.854457]
# lon_point_list = [4.377184, 13.407759, 14.435935, 2.349553, 4.377184]

# polygon_geom = Polygon(zip(lon_point_list, lat_point_list))
# crs = CRS("WGS84")
# polygon = gpd.GeoDataFrame(index=[0], crs=crs, geometry=[polygon_geom])       
# print(polygon.geometry)

# polygon.to_file(filename='polygon.geojson', driver='GeoJSON')
# polygon.to_file(filename='polygon.shp', driver="ESRI Shapefile")

In [14]:
# m = folium.Map([50.854457, 4.377184], zoom_start=5, tiles='cartodbpositron')
# folium.GeoJson(polygon).add_to(m)
# folium.LatLngPopup().add_to(m)
# m

## Loading solar eclipse path

In [15]:
eclipse_path = pd.read_csv('solar_eclipse_2024_04_08.csv')
eclipse_path.head()

,UTC,NLAT,NLON,SLAT,SLON,CLAT,CLON
0,16:42,05 30.6S,149 47.6W,06 11.7S,146 38.0W,05 50.2S,148 07.8W
1,16:44,04 20.5S,145 29.6W,05 08.4S,143 00.6W,04 44.0S,144 13.0W
2,16:46,03 21.2S,142 27.6W,04 12.3S,140 15.6W,03 46.4S,141 20.3W
3,16:48,02 27.1S,140 01.8W,03 20.2S,137 59.5W,02 53.3S,138 59.7W
4,16:50,01 36.2S,137 58.5W,02 30.8S,136 02.5W,02 03.3S,136 59.7W


In [16]:
# convert UTC to datetime
eclipse_path['UTC'] = pd.to_datetime('2024-04-08 ' + eclipse_path['UTC'], format='%Y-%m-%d %H:%M')


In [17]:
#function to convert DDM (degree decimal minutes) to decimal degrees

import re

def ddm_to_dec(ddm_str): 
    sign = -1 if re.search('[swSW]', ddm_str) else 1
    numbers = [*filter(len, re.split('\D+', ddm_str))]
    degree = numbers[0]
    minute_decimal = numbers[1] 
    decimal_val = numbers[2] if len(numbers) > 2 else '0' 
    minute_decimal += "." + decimal_val

    return sign * (int(degree) + float(minute_decimal) / 60)



In [18]:
eclipse_path.loc[:,'NLAT':'CLON'] = eclipse_path.loc[:,'NLAT':'CLON'].applymap(ddm_to_dec)

## Map the path using folium

In [19]:
lat_point_list = []
lon_point_list = []

lat_point_list.extend(eclipse_path['NLAT'].tolist())
lat_point_list.extend(eclipse_path['SLAT'].values[::-1].tolist())
lat_point_list.append(eclipse_path['NLAT'].iloc[0].tolist())

lon_point_list.extend(eclipse_path['NLON'].tolist())
lon_point_list.extend(eclipse_path['SLON'].values[::-1].tolist())
lon_point_list.append(eclipse_path['NLON'].iloc[0].tolist())

path_geom = Polygon(zip(lon_point_list, lat_point_list))
crs = CRS("WGS84")
path = gpd.GeoDataFrame(index=[0], crs=crs, geometry=[path_geom])       
print(path.geometry)

path.to_file(filename='path.geojson', driver='GeoJSON')
#polygon.to_file(filename='path.shp', driver="ESRI Shapefile")

0    POLYGON ((-149.79333 -5.51000, -145.49333 -4.3...
Name: geometry, dtype: geometry


In [10]:
path_map = folium.Map([39.0119, -98.4842], zoom_start=4, tiles='cartodbpositron')

folium.GeoJson(path).add_to(path_map)
folium.LatLngPopup().add_to(path_map)
path_map

## Creating boxes for selecting weather station data

In [20]:
box1_lat = [23.6, 28.2, 27.0, 22.4, 23.6]
box1_lon = [-106.9, -102.5, -101.1, -105.6, -106.9]

box2_lat = [27.0, 28.2, 32.9, 31.5, 27.0]
box2_lon = [-101.1, -102.5, -97.3, -96.1, -101.1]

box3_lat = [31.5, 32.9, 36.9, 35.5, 31.5]
box3_lon = [-96.1, -97.3, -91.9, -91.0, -96.1]

box1_geom = Polygon(zip(box1_lon, box1_lat))
crs = CRS("WGS84")
box1 = gpd.GeoDataFrame(index=[0], crs=crs, geometry=[box1_geom])

box2_geom = Polygon(zip(box2_lon, box2_lat))
crs = CRS("WGS84")
box2 = gpd.GeoDataFrame(index=[0], crs=crs, geometry=[box2_geom])

box3_geom = Polygon(zip(box3_lon, box3_lat))
crs = CRS("WGS84")
box3 = gpd.GeoDataFrame(index=[0], crs=crs, geometry=[box3_geom])

print(box3.geometry)

0    POLYGON ((-96.10000 31.50000, -97.30000 32.900...
Name: geometry, dtype: geometry


In [21]:
path_map = folium.Map([39.0119, -98.4842], zoom_start=4, tiles='cartodbpositron')

folium.GeoJson(path).add_to(path_map)
folium.GeoJson(box1).add_to(path_map)
folium.GeoJson(box2).add_to(path_map)
folium.GeoJson(box3).add_to(path_map)
folium.LatLngPopup().add_to(path_map)
path_map